Start by making sure you have the following packages in your environment:

In [25]:
# ! pip install evaluate datasets transformers[torch]

In [ ]:
MY_DATASET = "janaab/supreme-court-speech"

In [33]:
from datasets import load_dataset

sc_speech = load_dataset(
    MY_DATASET, split="test"
)

Using the latest cached version of the dataset since janaab/supreme-court-speech couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/janaab___supreme-court-speech/default/0.0.0/41c2679acab9b8ecb7f9d881e57d328c67d1e66c (last modified on Fri Aug 30 00:09:29 2024).


In [43]:
BASE_MODEL = "openai/whisper-small"
TUNED_MODEL = "janaab/whisper-small-sc"

In [19]:
# from transformers import GenerationConfig

# generation_config = GenerationConfig.from_pretrained("openai/whisper-small")

# generation_config.push_to_hub(TUNED_MODEL)

CommitInfo(commit_url='https://huggingface.co/janaab/whisper-small-dv/commit/1f395786f2fa19b936d99ce6d473dacaf487e464', commit_message='Upload config', commit_description='', oid='1f395786f2fa19b936d99ce6d473dacaf487e464', pr_url=None, pr_revision=None, pr_num=None)

In [44]:
from transformers import pipeline
import torch

if torch.cuda.is_available():
    device = "cuda:0"
    torch_dtype = torch.float16
else:
    device = "cpu"
    torch_dtype = torch.float32

pipe = pipeline(
    "automatic-speech-recognition",
    model=BASE_MODEL,
    torch_dtype=torch_dtype,
    device=device,
)

In [45]:
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

all_predictions = []

# run streamed inference
for prediction in tqdm(
    pipe(
        KeyDataset(sc_speech, "audio"),
        max_new_tokens=128,
        generate_kwargs={"task": "transcribe"},
        batch_size=32,
    ),
    total=len(sc_speech),
):
    all_predictions.append(prediction["text"])

100%|██████████| 4465/4465 [04:48<00:00, 15.48it/s]


In [46]:
from evaluate import load

wer_metric = load("wer")

wer_ortho = 100 * wer_metric.compute(
    references=sc_speech["transcript"], predictions=all_predictions
)
wer_ortho

47.670784616515924

In [47]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

# compute normalised WER
all_predictions_norm = [normalizer(pred) for pred in all_predictions]
all_references_norm = [normalizer(label) for label in sc_speech["transcript"]]

# filtering step to only evaluate the samples that correspond to non-zero references
all_predictions_norm = [
    all_predictions_norm[i]
    for i in range(len(all_predictions_norm))
    if len(all_references_norm[i]) > 0
]
all_references_norm = [
    all_references_norm[i]
    for i in range(len(all_references_norm))
    if len(all_references_norm[i]) > 0
]

wer = 100 * wer_metric.compute(
    references=all_references_norm, predictions=all_predictions_norm
)

wer

35.21903104148535